In [1]:
# install anvil and import anvil server
# !pip install anvil-uplink
import anvil.server

In [2]:
# !pip install PyMySQL
import pymysql

In [3]:
# !pip install plotly==5.6.0
from datetime import datetime
import plotly.express as px
import pandas as pd

In [4]:
# Connect to the anvil frontend
anvil.server.connect("PVI2LD3VLGMNLT22SC6MILQG-FX3COW6HW5E3NGBT")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [5]:
def set_connection():
#     db = pymysql.connect(host='sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com',
#                          port =29563,user='capstone',password='capstone123!!!',
#                          db='circle_database_version3')
    db = pymysql.connect(host='localhost',
                         port =3306,user='root',password='htdhylyzhh',
                         db='circle_database_version3')
    return db

In [6]:
def stop_connection(db):
    db.close()

In [7]:
def generate_query(vehicle_id, start_time, end_time, lower_constraint, upper_constraint, table, kind):
    query = "SELECT " + kind + " , elapsed_time FROM " + table + " WHERE elapsed_time > '" + str(start_time) + "' and elapsed_time < '" + str(end_time) + "' and drive_id = '" + str(vehicle_id) + "' and " + kind + " > " + lower_constraint + " and " + kind + " < " + upper_constraint
    return query

In [8]:
def datetime_to_float(d):
    return d.timestamp()

In [9]:
@anvil.server.callable
def query_lead_distance(vehicle_id, start_time, end_time, lower_lead, upper_lead):
    start_time = start_time.replace(tzinfo=None)
    end_time = end_time.replace(tzinfo=None)
    MAXIMUM = 10000
    lower_lead = str(lower_lead) if lower_lead != "" else str(- MAXIMUM)        
    upper_lead = str(upper_lead) if upper_lead != "" else str(MAXIMUM)  
        
    query = generate_query(vehicle_id, start_time, end_time, lower_lead, upper_lead, "fact_lead_distance", "lead_distance")
    db = set_connection()  
    cursor = db.cursor()
    cursor.execute(query)
    data = cursor.fetchall()

    stop_connection(db)
    return list(data)

@anvil.server.callable
def query_speed(vehicle_id, start_time, end_time, lower_speed, upper_speed):
    start_time = start_time.replace(tzinfo=None)
    end_time = end_time.replace(tzinfo=None)
    MAXIMUM = 10000
    lower_speed = str(lower_speed) if lower_speed != "" else str(- MAXIMUM)        
    upper_speed = str(upper_speed) if upper_speed != "" else str(MAXIMUM)  
        
    query = generate_query(vehicle_id, start_time, end_time, lower_speed, upper_speed, "fact_speed", "speed")
    
    db = set_connection()  
    
    cursor = db.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    
    stop_connection(db)
    
    return list(data)

@anvil.server.callable
def query_acc(vehicle_id, start_time, end_time, lower_acceleration, upper_acceleration):
    st = start_time.replace(tzinfo=None)
    et = end_time.replace(tzinfo=None)
    MAXIMUM = 10000 
    lower_acceleration = str(lower_acceleration) if lower_acceleration != "" else str(- MAXIMUM)        
    upper_acceleration = str(upper_acceleration) if upper_acceleration != "" else str(MAXIMUM)
    query = generate_query(vehicle_id, start_time, end_time, lower_acceleration, upper_acceleration, "fact_acc", "fact_acc")
    
    db = set_connection() 
    
    cursor = db.cursor()
    cursor.execute(query)
    data = cursor.fetchall()

    stop_connection(db)
    return list(data)

In [10]:
@anvil.server.callable
def query_all(drive_id, start_time, end_time):
    if start_time != None and end_time != None:
        start_time = start_time.replace(tzinfo=None)
        end_time = end_time.replace(tzinfo=None)
        start_time = datetime_to_float(start_time)
        end_time = datetime_to_float(end_time)
        query = "Select elapsed_time, speed, lead_distance, acceleration FROM fact_drive_data WHERE drive_id = " + str(drive_id) + " and elapsed_time > " + str(start_time) + " and elapsed_time < " + str(end_time)
        db = set_connection() 
        df = pd.read_sql(query, db)
    else:
        query = "Select elapsed_time, speed, lead_distance, acceleration FROM fact_drive_data WHERE drive_id = " + str(drive_id)
        db = set_connection() 
        df = pd.read_sql(query, db)
    if len(df["elapsed_time"].tolist()) == 0:
        return None, None, None, None
    return df["elapsed_time"].to_list(), df["lead_distance"].to_list(), df["speed"].to_list(), df["acceleration"].to_list()

In [11]:
@anvil.server.callable
# 记得改17行，等数据库好了之后
def query_drive_id_by_gps(lat, long):
    RANGE = 5
    query_id = "Select drive_id FROM fact_gps_message where latitude > " + str(float(lat)-RANGE) + " and latitude < " + str(float(lat)+RANGE) + " and longitude < " + str(float(long)+RANGE) + " and longitude > " + str(float(long)-RANGE)
    
    db = set_connection() 
    cursor = db.cursor()
    cursor.execute(query_id)
    drive_ids = list(set(cursor.fetchall()))
    string_list = []
    gps_list = []
    for drive_id in drive_ids:
        cleaned_drive_id = drive_id[0]
        query = "Select vin FROM dim_vehicle WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        vin = cursor.fetchall()
#         if vin[0] != None:
#             vin = vin[0][0]
        print(vin)
        query = "Select systime_start, latitude_start, longitude_start FROM dim_drive WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        l = cursor.fetchall()[0]
        if l[0] != None:
            time = datetime.fromtimestamp(l[0])
            latitude = l[1]
            longitude = l[2]
            string_list.append("Vehicle ID: " + str(vin) + " Start Time: " + str(time)[:19] + " Drive ID: " + str(cleaned_drive_id))
            gps_list.append([latitude, longitude])
        if l[0] == None:
            string_list.append("Vehicle ID: " + str(vin) + " Start Time: " + "Not Available" + " Drive ID: " + str(cleaned_drive_id))
            latitude = l[1]
            longitude = l[2]
            gps_list.append([latitude, longitude])
    stop_connection(db)
    return string_list, gps_list

In [12]:
@anvil.server.callable
def query_drive_id_by_gps_and_time(lat, long, start_time, end_time):
    RANGE = 5
    start_time = start_time.replace(tzinfo=None)
    end_time = end_time.replace(tzinfo=None)
    start_time = datetime_to_float(start_time)
    end_time = datetime_to_float(end_time)
    query_id = "Select drive_id FROM fact_gps_message where latitude > " + str(float(lat)-RANGE) + " and latitude < " + str(float(lat)+RANGE) + " and longitude < " + str(float(long)+RANGE) + " and longitude > " + str(float(long)-RANGE) + " and systime > " + str(start_time) + " and systime < " + str(end_time)
    db = set_connection() 
    cursor = db.cursor()
    cursor.execute(query_id)
    drive_ids = list(set(cursor.fetchall()))
    string_list = []
    gps_list = []
    for drive_id in drive_ids:
        cleaned_drive_id = drive_id[0]
        query = "Select vin FROM dim_vehicle WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        vin = cursor.fetchall()[0][0]
        query = "Select systime_start, latitude_start, longitude_start FROM dim_drive WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        l = cursor.fetchall()[0]
        if l[0] != None:
            time = datetime.fromtimestamp(l[0])
            latitude = l[1]
            longitude = l[2]
            string_list.append("Vehicle ID: " + str(vin) + " Start Time: " + str(time)[:19] + " Drive ID: " + str(cleaned_drive_id))
            gps_list.append([latitude, longitude])
        if l[0] == None:
            string_list.append("Vehicle ID: " + str(vin) + " Start Time: " + "Not Available" + " Drive ID: " + str(cleaned_drive_id))
            latitude = l[1]
            longitude = l[2]
            gps_list.append([latitude, longitude])
    stop_connection(db)
    return string_list, gps_list

In [13]:
@anvil.server.callable
def query_drive_id_by_vehicle(vehicle_id):
    RANGE = 500
    query_id = "Select drive_id FROM fact_gps_message where latitude > " + str(float(lat)-RANGE) + " and latitude < " + str(float(lat)+RANGE) + " and longitude < " + str(float(long)+RANGE) + " and longitude > " + str(float(long)-RANGE)
    
    db = set_connection() 
    cursor = db.cursor()
    cursor.execute(query_id)
    drive_ids = list(set(cursor.fetchall()))
    string_list = []
    gps_list = []
    for drive_id in drive_ids:
        cleaned_drive_id = drive_id[0]
        query = "Select vin FROM dim_vehicle WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        vin = cursor.fetchall()[0][0]
        query = "Select systime_start, latitude_start, longitude_start FROM dim_drive WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        l = cursor.fetchall()[0]
        if l[0] != None:
            time = datetime.fromtimestamp(l[0])
            latitude = l[1]
            longitude = l[2]
            string_list.append("Vehicle ID: " + str(vin) + " Start Time: " + str(time)[:19] + " Drive ID: " + str(cleaned_drive_id))
            gps_list.append([latitude, longitude])
        if l[0] == None:
            string_list.append("Vehicle ID: " + str(vin) + " Start Time: " + "Not Available" + " Drive ID: " + str(cleaned_drive_id))
            latitude = l[1]
            longitude = l[2]
            gps_list.append([latitude, longitude])
    stop_connection(db)
    return string_list, gps_list

In [14]:
@anvil.server.callable
def query_routine(drive_id):
    query_gps = "Select latitude, longitude FROM fact_gps_message where drive_id = " + str(drive_id)
    
    db = set_connection() 
    cursor = db.cursor()
    cursor.execute(query_gps)
    gps_data = cursor.fetchall()
    
    gps = []
    for latlong in gps_data:
        gps.append([latlong[0], latlong[1]])
    stop_connection(db)
    return gps

In [15]:
@anvil.server.callable
def query_drive_id_by_time(start_time, end_time):
    start_time = start_time.replace(tzinfo=None)
    end_time = end_time.replace(tzinfo=None)
    start_time = datetime_to_float(start_time)
    end_time = datetime_to_float(end_time)
    query_id = "Select vehicle_id FROM dim_drive WHERE systime_start > " + str(start_time) + " and systime_end < " + str(end_time)
    db = set_connection() 
    cursor = db.cursor()
    cursor.execute(query_id)
    drive_ids = list(cursor.fetchall())
    string_list = []
    gps_list = []
    for drive_id in drive_ids:
        cleaned_drive_id = drive_id[0]
        query = "Select vin FROM dim_vehicle WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        vin = cursor.fetchall()[0][0]
        query = "Select systime_start, latitude_start, longitude_start FROM dim_drive WHERE id = " + str(cleaned_drive_id)     
        cursor.execute(query)
        l = cursor.fetchall()[0]
        if l[0]!=None:
            time = datetime.fromtimestamp(float(l[0]))
            print(float(l[0]))
            latitude = l[1]
            longitude = l[2]
            string_list.append("Vehicle ID: " + str(vin) + " Start Time: " + str(time)[:19] + " Drive ID: " + str(cleaned_drive_id))
            gps_list.append([latitude, longitude])
        if l[0] == None:
            string_list.append("Vehicle ID: " + str(vin) + " Start Time: " + "Not Available" + " Drive ID: " + str(cleaned_drive_id))
            latitude = l[1]
            longitude = l[2]
            gps_list.append([latitude, longitude])
    stop_connection(db)
    return string_list, gps_list

In [16]:
@anvil.server.callable
def generate_CSV(drive_id):
    query = "Select latitude, longitude, altitude, speed, lead_distance, acceleration, relative_vel, acc_status FROM fact_drive_data WHERE drive_id = " + str(drive_id)
    db = set_connection() 
    df = pd.read_sql(query, db)
    df.to_csv('search.csv')

In [17]:
@anvil.server.callable
def general_query(drive_id, lower_speed, upper_speed, lower_lead_distance, upper_lead_distance, lower_acceleration, upper_acceleration):
    MAXIMUM = 500000
    lower_speed = - MAXIMUM if lower_speed == "" else lower_speed
    upper_speed = MAXIMUM if upper_speed == "" else upper_speed
    lower_lead_distance =  -MAXIMUM if lower_lead_distance == "" else lower_lead_distance
    upper_lead_distance = MAXIMUM if upper_lead_distance == "" else upper_lead_distance
    lower_acceleration = -MAXIMUM if lower_acceleration == "" else lower_acceleration
    upper_acceleration = MAXIMUM if upper_acceleration == "" else upper_acceleration
    query = "Select elapsed_time, speed, lead_distance, acceleration FROM fact_drive_data WHERE drive_id = " + str(drive_id) + " and speed > " + str(lower_speed) + " and speed < " + str(upper_speed) + " and lead_distance > " + str(lower_lead_distance) + " and lead_distance < " + str(upper_lead_distance) + " and acceleration > " + str(lower_acceleration) + " and acceleration < " + str(upper_acceleration)
    db = set_connection() 
    df = pd.read_sql(query, db)
    if len(df["elapsed_time"].tolist()) == 0:
        return None, None, None, None
    return df["elapsed_time"].to_list(), df["lead_distance"].to_list(), df["speed"].to_list(), df["acceleration"].to_list()

In [18]:
@anvil.server.callable
def query_drive_id_by_constrains(lower_speed, upper_speed, lower_lead_distance, upper_lead_distance, lower_acceleration, upper_acceleration):
    MAXIMUM = 500000
    lower_speed = - MAXIMUM if lower_speed == "" else lower_speed
    upper_speed = MAXIMUM if upper_speed == "" else upper_speed
    lower_lead_distance =  -MAXIMUM if lower_lead_distance == "" else lower_lead_distance
    upper_lead_distance = MAXIMUM if upper_lead_distance == "" else upper_lead_distance
    lower_acceleration = -MAXIMUM if lower_acceleration == "" else lower_acceleration
    upper_acceleration = MAXIMUM if upper_acceleration == "" else upper_acceleration
    query_id = "Select drive_id FROM fact_drive_data WHERE speed > " + str(lower_speed) + " and speed < " + str(upper_speed) + " and lead_distance > " + str(lower_lead_distance) + " and lead_distance < " + str(upper_lead_distance) + " and acceleration > " + str(lower_acceleration) + " and acceleration < " + str(upper_acceleration)
    db = set_connection() 
    cursor = db.cursor()
    cursor.execute(query_id)
    drive_ids = list(set(cursor.fetchall()))
    gps_list = []
    string_list = []
    prefix_list = []
    for drive_id in drive_ids:
        cleaned_drive_id = drive_id[0]
        query = "Select vin FROM dim_vehicle WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        vin = cursor.fetchall()[0][0]
        query = "Select systime_start, latitude_start, longitude_start, file_prefix FROM dim_drive WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        l = cursor.fetchall()[0]
        if l[0] != None:
            time = datetime.fromtimestamp(float(l[0]))
            latitude = l[1]
            longitude = l[2]
            file_prefix = l[3]
            string_list.append("Vehicle ID: " + str(vin) + " Start Time: " + str(time)[:19] + " Drive ID: " + str(cleaned_drive_id))
            gps_list.append([latitude, longitude])
            prefix_list.append(file_prefix)
        if l[0] == None:
            string_list.append("Vehicle ID: " + str(vin) + " Start Time: " + "Not Available" + " Drive ID: " + str(cleaned_drive_id))
            latitude = l[1]
            longitude = l[2]
            file_prefix = l[3]
            gps_list.append([latitude, longitude])
            prefix_list.append(file_prefix)
    stop_connection(db)
    return string_list, gps_list, prefix_list

In [22]:
@anvil.server.callable
def event_search_by_speed(lower, upper, time):
    query = "WITH windows AS ( SELECT *, \
        MIN(speed) OVER (PARTITION BY drive_id ORDER BY elapsed_time ROWS BETWEEN " + str(5 * time) + " PRECEDING AND " + str(5 * time) + " FOLLOWING) AS min_speed, \
        MAX(speed) OVER (PARTITION BY drive_id ORDER BY elapsed_time ROWS BETWEEN " + str(5 * time) + " PRECEDING AND " + str(5 * time) + " FOLLOWING) AS max_speed \
        FROM fact_drive_data) \
    SELECT drive_id, elapsed_time FROM windows WHERE min_speed <= " + str(lower) + " AND max_speed >= " + str(upper)
    print(query)
    string_list = []
    db = set_connection() 
    cursor = db.cursor()
    df = pd.read_sql(query, db)
    if len(df) == 0:
        return None
    else:
        for i in range(len(df)):
            drive_id = df.iloc[i]["drive_id"]
            timestamp = datetime.fromtimestamp(float(df.iloc[i]["elapsed_time"]) - float(time)/2)
            elapsed_time = float(df.iloc[i]["elapsed_time"]) - float(time)/2
            query_vin = "Select vin FROM dim_vehicle WHERE id = " + str(drive_id)
            cursor.execute(query_vin)
            vin = cursor.fetchall()[0][0]
            string_list.append("Vehicle ID: " + str(vin) + " Time: " + str(timestamp) + " Time Stamp: " + str(elapsed_time) + " Drive ID: " + str(drive_id))
    stop_connection(db)
    
    return string_list


@anvil.server.callable
def event_search_by_acc(lower, upper, time):
    query = "WITH windows AS ( SELECT *, \
        MIN(acceleration) OVER (PARTITION BY drive_id ORDER BY elapsed_time ROWS BETWEEN " + str(5 * time) + " PRECEDING AND " + str(5 * time) + " FOLLOWING) AS min_acceleration, \
        MAX(acceleration) OVER (PARTITION BY drive_id ORDER BY elapsed_time ROWS BETWEEN " + str(5 * time) + " PRECEDING AND " + str(5 * time) + " FOLLOWING) AS max_acceleration \
        FROM fact_drive_data) \
    SELECT drive_id, elapsed_time FROM windows WHERE min_acceleration <= " + str(lower) + " AND max_acceleration >= " + str(upper)
    print(query)
    string_list = []
    db = set_connection() 
    cursor = db.cursor()
    df = pd.read_sql(query, db)
    if len(df) == 0:
        return None
    else:
        for i in range(len(df)):
            drive_id = df.iloc[i]["drive_id"]
            timestamp = datetime.fromtimestamp(float(df.iloc[i]["elapsed_time"]) - float(time)/2)
            elapsed_time = float(df.iloc[i]["elapsed_time"]) - float(time)/2
            query_vin = "Select vin FROM dim_vehicle WHERE id = " + str(drive_id)
            cursor.execute(query_vin)
            vin = cursor.fetchall()[0][0]
            string_list.append("Vehicle ID: " + str(vin) + " Time: " + str(timestamp) + " Time Stamp: " + str(elapsed_time) + " Drive ID: " + str(drive_id))
    stop_connection(db)
    return string_list

@anvil.server.callable
def event_search_by_lead_distance(lower, upper, time):
    query = "WITH windows AS ( SELECT *, \
        MIN(lead_distance) OVER (PARTITION BY drive_id ORDER BY elapsed_time ROWS BETWEEN " + str(5 * time) + " PRECEDING AND " + str(5 * time) + " FOLLOWING) AS min_lead_distance, \
        MAX(lead_distance) OVER (PARTITION BY drive_id ORDER BY elapsed_time ROWS BETWEEN " + str(5 * time) + " PRECEDING AND " + str(5 * time) + " FOLLOWING) AS max_lead_distance \
        FROM fact_drive_data) \
    SELECT drive_id, elapsed_time FROM windows WHERE min_lead_distance <= " + str(lower) + " AND max_lead_distance >= " + str(upper)
    string_list = []
    db = set_connection() 
    cursor = db.cursor()
    df = pd.read_sql(query, db)
    if len(df) == 0:
        return None
    else:
        for i in range(len(df)):
            drive_id = df.iloc[i]["drive_id"]
            timestamp = datetime.fromtimestamp(float(df.iloc[i]["elapsed_time"]) - float(time)/2)
            elapsed_time = float(df.iloc[i]["elapsed_time"]) - float(time)/2
            query_vin = "Select vin FROM dim_vehicle WHERE id = " + str(drive_id)
            cursor.execute(query_vin)
            vin = cursor.fetchall()[0][0]
            string_list.append("Vehicle ID: " + str(vin) + " Time: " + str(timestamp) + " Time Stamp: " + str(elapsed_time) + " Drive ID: " + str(drive_id))
    stop_connection(db)
    return string_list


In [23]:
@anvil.server.callable
def event_search_by_id_and_time(time, drive_id, elapsed_time):
    query = "SELECT speed, acceleration, lead_distance, elapsed_time FROM fact_drive_data WHERE drive_id = " + str(drive_id) + " AND elapsed_time <= " + str(float(elapsed_time) + float(time)/2) + " AND elapsed_time >= " + str(float(elapsed_time) - float(time)/2)
    print(query)
    db = set_connection() 
    df = pd.read_sql(query, db)
    stop_connection(db)
    return df["elapsed_time"].to_list(), df["lead_distance"].to_list(), df["speed"].to_list(), df["acceleration"].to_list()   

In [21]:
anvil.server.wait_forever()

,speed,acceleration,lead_distance,elapsed_time
0,0,-0.44,252,1617722252.1
1,0,-0.4,252,1617722252.2
2,0,-0.07,252,1617722252.3
3,0,-0.01,252,1617722252.4
4,0,0,252,1617722252.5
...,...,...,...,...
565,5.57,-0.78,252,1617722311.6
566,5.29,-0.78,252,1617722311.7
567,4.78,-0.72,252,1617722311.9
568,4.5,-0.59,252,1617722312


KeyboardInterrupt: 

WITH windows AS ( SELECT *,         MIN(speed) OVER (PARTITION BY drive_id ORDER BY elapsed_time ROWS BETWEEN 50 PRECEDING AND 50 FOLLOWING) AS min_speed,         MAX(speed) OVER (PARTITION BY drive_id ORDER BY elapsed_time ROWS BETWEEN 50 PRECEDING AND 50 FOLLOWING) AS max_speed         FROM fact_drive_data)     SELECT drive_id, elapsed_time FROM windows WHERE min_speed <= 0 AND max_speed >= 5


KeyboardInterrupt: 